In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
from copy import deepcopy

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

In [34]:
df = pd.read_csv('../Data Sources/gun_violence_archive_github_download.csv')

original_incident_ids = list(df['incident_id'])
with open('../Pickles/original_incident_ids.pkl', 'wb') as f:
    pkl.dump(original_incident_ids, f)
    
df['participant_type'] = df['participant_type'].replace('\|', ':', regex = True).str.split(":")
df['participant_status'] = df['participant_status'].replace('\|', ':', regex = True).str.split(":")
df['participant_gender'] = df['participant_gender'].replace('\|', ':', regex = True).str.split(":")
df['participant_age'] = df['participant_age'].replace('\|', ':', regex = True).str.split(":")

df['scraped'] = False

df.rename({'n_guns_involved': 'num_guns_involved'}, axis = 1, inplace = True)
df.drop(['n_killed', 'n_injured', 'source_url', 'incident_url_fields_missing'], axis = 1, inplace = True)

In [35]:
len(df)

239677

In [36]:
df_scraped = pd.read_csv('../Data Sources/gun_violence_archive_scraped_full_2.csv')

print(df_scraped['page_response_code'].value_counts())
df_scraped = deepcopy(df_scraped[df_scraped['page_response_code']==200].reset_index())
df_scraped['page_response_code'] = df_scraped['page_response_code'].astype(int)
df_scraped.drop(['index', 'page_response_code', 'Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

df_scraped['participant_status'] = df_scraped['participant_status'].replace(", A", ",A", regex = True)
df_scraped['participant_status'] = df_scraped['participant_status'].replace(", U", ",U", regex = True)
df_scraped['participant_status'] = df_scraped['participant_status'].replace(", K", ",K", regex = True)
df_scraped['participant_status'] = df_scraped['participant_status'].replace(", I", ",I", regex = True)
for column in df_scraped.columns:
    if 'participant' in column:
        df_scraped[column] = df_scraped[column].replace("'", '', regex = True)
        df_scraped[column] = df_scraped[column].str.split(', ')

df_scraped['scraped'] = True

df = deepcopy(pd.concat([df, df_scraped], sort=False))

200.00    60540
Name: page_response_code, dtype: int64


In [37]:
len(df)

300217

In [39]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['date', 'incident_id', 'scraped'], ascending=True)

In [41]:
df.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'address',
       'incident_url', 'congressional_district', 'gun_stolen', 'gun_type',
       'incident_characteristics', 'latitude', 'location_description',
       'longitude', 'num_guns_involved', 'notes', 'participant_age',
       'participant_age_group', 'participant_gender', 'participant_name',
       'participant_relationship', 'participant_status', 'participant_type',
       'sources', 'state_house_district', 'state_senate_district', 'scraped'],
      dtype='object')

In [42]:
before = len(df)
####################################################################################
df = deepcopy(df[df['participant_type'].isnull()==False].reset_index())
df.drop('index', axis = 1, inplace = True)
####################################################################################
after = len(df)
print(str(before-after) + " rows removed from removing null participant_type rows.")

before = len(df)
####################################################################################
duplicate_list = ['incident_id']
df.drop_duplicates(subset = duplicate_list, keep = 'last', inplace = True)
####################################################################################
after = len(df)
print(str(before-after) + " rows removed from dropping literal duplicates.")

before = len(df)
####################################################################################
duplicate_list = ['date', 'state', 'city_or_county', 'address', 'incident_characteristics', 'notes']
df.drop_duplicates(subset = duplicate_list, keep = 'last', inplace = True)
####################################################################################
after = len(df)
print(str(before-after) + " rows removed from dropping suspected duplicates. (Duplicate pages with different incident IDs.)\n\t\t\t\t\t\t   This occurs when GVA deletes a pages and reposts it.\n")

# how many of our scraped pages fall in the id range of our original pages?
# value was 251 on 4/20, beginning of backtrack scrape through original id range
print(str(len(df[(df['incident_id'] >= int(df[df['scraped']==False]['incident_id'].min())) & (df['incident_id'] <= int(df[df['scraped']==False]['incident_id'].max())) & (df['scraped']==True)])) + " scraped pages within the id range of our original pages.")
# value was 1,040 on 4/20, beginning of backtrack scrape through original id range
print(str(len(df[(df['date'] >= df[df['scraped']==False]['date'].min()) & (df['date'] <= df[df['scraped']==False]['date'].max()) & (df['scraped']==True)])) + " scraped pages within the date range of our original pages.")

981 rows removed from dropping literal duplicates.
523 rows removed from dropping suspected duplicates. (Duplicate pages with different incident IDs.)
						   This occurs when GVA deletes a pages and reposts it.

392 scraped pages within the id range of our original pages.
1229 scraped pages within the date range of our original pages.


In [10]:
null_columns = df.columns[df.isnull().any()]
print(df[null_columns].isnull().sum())

address                      18345
congressional_district       12172
gun_stolen                   90216
gun_type                     90184
incident_characteristics       277
latitude                      8506
location_description        221090
longitude                     8506
num_guns_involved            90184
notes                        91183
participant_age              67365
participant_age_group        17236
participant_gender           11477
participant_name             97320
participant_relationship    198907
participant_status            2762
sources                        381
state_house_district         36419
state_senate_district        29697
dtype: int64


In [11]:
# null_participant_status_ids = list(set(df[df['participant_status'].isnull()]['incident_id']))

# with open('null_participant_status_ids.pkl', 'wb') as f:
#     pkl.dump(null_participant_status_ids, f)

In [33]:
len(df[df['participant_status'].isnull()])

2762

In [34]:
# df.loc[(['Killed' not in s and 'Injured' not in s and 'Unharmed' not in s and 'Arrested' not in s for s in df['participant_status'].astype(str)]), 'participant_status'] = None

In [35]:
# len(df[df['participant_status'].isnull()])

3190

In [36]:
df['incident_characteristics'] = df['incident_characteristics'].astype(str)
# assertion that null participant_status values are only in the original scraped file
# value will be 2 if this fails
assert len(df[df['participant_status'].isnull()]['scraped'].value_counts().index)==1
# Using incident_characteristics to infer participant_status
# assumption for general killed/injury language is that it refers to the victim
# this assumption is improved below improved where conditionals from both killed and both injured
# go to the other list when there are only one recorded participant type
# example: injury/death is for victim, but if only suspects are recorded,
# it moves to the other list and says the subject died instead
null_status_victim_killed = list(df.loc[
    (df['participant_status'].isnull())
    & (['Non-Shooting Incident' not in s for s in df['incident_characteristics']])
    & ((['injury/death' in s and 'no injury/death' not in s for s in df['incident_characteristics']])
       or (['Shot - Dead' in s for s in df['incident_characteristics']]))]['incident_id'].astype(int))

null_status_suspect_killed = list(df.loc[
    (df['participant_status'].isnull())
    & (['Non-Shooting Incident' not in s for s in df['incident_characteristics']])
    & ((['subject/suspect/perpetrator killed' in s for s in df['incident_characteristics']])
       or (['Suicide^' in s for s in df['incident_characteristics']]))]['incident_id'].astype(int))

null_status_victim_injured = list(df.loc[
    (df['participant_status'].isnull())
    & (['Non-Shooting Incident' not in s for s in df['incident_characteristics']])
    & ((['injured' in s or 'Injured' in s for s in df['incident_characteristics']])
       or (['wounded' in s or 'Wounded' in s for s in df['incident_characteristics']])
       or (['Accidental Shooting - Injury' in s for s in df['incident_characteristics']]))]['incident_id'].astype(int))

null_status_suspect_injured = list(df.loc[
    (df['participant_status'].isnull())
    & (['Non-Shooting Incident' not in s for s in df['incident_characteristics']])
    & ((['subject/suspect/perpetrator shot' in s or 'subject/suspect/perpetrator injured' in s for s in df['incident_characteristics']]))]['incident_id'].astype(int))

print(str(len(df[df['participant_status'].isnull()])) + " total pages are missing participant_status values.")
print("\nFilling in victims killed for " + str(len(null_status_victim_killed)) + " initial pages.")
print("Filling in victims injured for " + str(len(null_status_victim_injured)) + " initial pages.")
print("Filling in suspects killed for " + str(len(null_status_suspect_killed)) + " initial pages.")
print("Filling in suspects injured for " + str(len(null_status_suspect_injured)) + " initial pages.\n")

suspect_id_list = list(set(df[(['Subject-Suspect' in s for s in df['participant_type']])]['incident_id'].astype(int)))
victim_id_list = list(set(df[(['Victim' in s for s in df['participant_type']])]['incident_id'].astype(int)))

filtered_list = list(set(null_status_victim_killed) - set(victim_id_list))
print(str(len(filtered_list)) + " pages where victims were inferred to have been killed don't list victims.")
if len(filtered_list)>0:
    null_status_suspect_killed.extend(filtered_list)

filtered_list = list(set(null_status_victim_injured) - set(victim_id_list))
print(str(len(filtered_list)) + " pages where victims were inferred to have been injured don't list victims.")
if len(filtered_list)>0:
    null_status_suspect_injured.extend(filtered_list)

filtered_list = list(set(null_status_suspect_killed) - set(suspect_id_list))
print(str(len(filtered_list)) + " pages where suspects were inferred to have been killed don't list suspects.")
if len(filtered_list)>0:
    null_status_victim_killed.extend(filtered_list)

filtered_list = list(set(null_status_suspect_injured) - set(suspect_id_list))
print(str(len(filtered_list)) + " pages where suspects were inferred to have been injured don't list suspects.")
if len(filtered_list)>0:
    null_status_victim_injured.extend(filtered_list)

print("\nFilling in victims killed for " + str(len(null_status_victim_killed)) + " pages.")
print("Filling in victims injured for " + str(len(null_status_victim_injured)) + " pages.")
print("Filling in suspects killed for " + str(len(null_status_suspect_killed)) + " pages.")
print("Filling in suspects injured for " + str(len(null_status_suspect_injured)) + " pages.")
print("\nCannot infer missing participant_status for " + str(len(df[(df['participant_status'].isnull()) & (['Accidental Shooting' in s for s in df['incident_characteristics']])])) + " pages.")

KeyError: 'incident_characteristics'

In [13]:
df['gun_stolen'] = df['gun_stolen'].astype(str)
df['participant_relationship'] = df['participant_relationship'].astype(str)

df.loc[(['Stolen' in s for s in df['gun_stolen']]) or
       (['stolen' in s for s in df['gun_stolen']]) or
       (['Gun(s) stolen' in s for s in df['incident_characteristics']]) or
       (['Stolen/Illegally owned gun{s}' in s for s in df['incident_characteristics']]) or
       (['stolen gun' in s for s in df['incident_characteristics']]), 'gun_stolen'] = 1
df.loc[df['gun_stolen']!=1, 'gun_stolen'] = 0
df.loc[(['Suicide' in s for s in df['incident_characteristics']]) or
       (['suicide' in s for s in df['incident_characteristics']]), 'suicide'] = 1
df.loc[df['suicide']!=1, 'suicide'] = 0
df.loc[(['Accidental' in s for s in df['incident_characteristics']]) or
       (['accidental' in s for s in df['incident_characteristics']]), 'accidental'] = 1
df.loc[df['accidental']!=1, 'accidental'] = 0
df.loc[(['Domestic Violence' in s for s in df['incident_characteristics']]), 'domestic_violence'] = 1
df.loc[(['Domestic Violence' not in s for s in df['incident_characteristics']]), 'domestic_violence'] = 0
df.loc[(['Gang' in s for s in df['participant_relationship']]) or
       (['Gang' in s for s in df['incident_characteristics']]), 'gang_related'] = 1
df.loc[df['gang_related']!=1, 'gang_related'] = 0
df.loc[(['Family' in s for s in df['participant_relationship']]) or
       (['Family' in s for s in df['incident_characteristics']]), 'family_related'] = 1
df.loc[df['family_related']!=1, 'family_related'] = 0
df.loc[(['Non-Shooting Incident' in s for s in df['incident_characteristics']]), 'non_shooting_incident'] = 1
df.loc[df['non_shooting_incident']!=1, 'non_shooting_incident'] = 0
df.loc[(['Gun shop robbery' in s for s in df['incident_characteristics']]), 'gun_shop_robbery'] = 1
df.loc[df['gun_shop_robbery']!=1, 'gun_shop_robbery'] = 0
df.loc[(['Drive-by' in s for s in df['incident_characteristics']]), 'drive_by_shooting'] = 1
df.loc[df['drive_by_shooting']!=1, 'drive_by_shooting'] = 0
df.loc[(['Officer Involved Shooting' in s for s in df['incident_characteristics']]), 'officer_involved_shooting'] = 1
df.loc[df['officer_involved_shooting']!=1, 'officer_involved_shooting'] = 0
# df.loc[(['Child' in s for s in df['incident_characteristics']]), 'child_involved'] = 1
# df.loc[df['child_involved']!=1, 'child_involved'] = 0
df.loc[(['Mass Shooting' in s for s in df['incident_characteristics']]), 'mass_shooting'] = 1
df.loc[df['mass_shooting']!=1, 'mass_shooting'] = 0
df.loc[(['Drug' in s for s in df['incident_characteristics']]), 'drug_involved'] = 1
df.loc[df['drug_involved']!=1, 'drug_involved'] = 0
df.loc[(['Assault weapon' in s for s in df['incident_characteristics']]), 'assault_weapon'] = 1
df.loc[df['assault_weapon']!=1, 'assault_weapon'] = 0

df.drop(['address', 'incident_url', 'sources',
         'incident_characteristics', 'participant_name',
         'participant_gender', 'participant_relationship',
         'gun_type', 'participant_age_group',
         'location_description', 'notes'], axis = 1, inplace = True)

In [14]:
boolean_columns = ['gun_stolen', 'suicide', 'accidental', 'domestic_violence',
                   'gang_related', 'family_related', 'non_shooting_incident', 'gun_shop_robbery',
                   'drive_by_shooting', 'officer_involved_shooting', #'child_involved',
                   'mass_shooting', 'drug_involved', 'assault_weapon']

for column in boolean_columns:
    df[column] = df[column].astype(int)
    print("{}: ".format(column) + str(len(df[df[column]==1])))

gun_stolen: 7361
suicide: 7533
accidental: 9738
domestic_violence: 14089
gang_related: 316
family_related: 4336
non_shooting_incident: 50547
gun_shop_robbery: 567
drive_by_shooting: 15512
officer_involved_shooting: 17489
mass_shooting: 2019
drug_involved: 21764
assault_weapon: 2851


In [15]:
assert len(df['state'].unique())==51

In [16]:
df.to_pickle('../Pickles/df1.pkl')

In [7]:
df = pd.read_pickle('../Pickles/df1.pkl')

In [8]:
len(df)

268677

In [19]:
# participant type
part_type_df = df.set_index(['incident_id'])['participant_type'].apply(pd.Series).stack().reset_index(level=1, drop=True)
part_type_df = part_type_df.reset_index()
part_type_df.columns = ['incident_id','participant_type']
part_type_df = part_type_df[(part_type_df['participant_type']=='Victim') | (part_type_df['participant_type']=='Subject-Suspect')]
part_type_df['participant_index'] = part_type_df.groupby('incident_id').cumcount()
part_type_df = part_type_df.reset_index()
part_type_df.drop(['index'], axis = 1, inplace = True)

# participant age
part_age_df = df.set_index(['incident_id'])['participant_age'].apply(pd.Series).stack().reset_index(level=1, drop=True)
part_age_df = part_age_df.reset_index()
part_age_df.columns = ['incident_id','participant_age']
# added code below due to scraping inconsistency
part_age_df = deepcopy(pd.merge(part_age_df, df[['incident_id', 'scraped']], on = 'incident_id', how='left'))
part_age_df.loc[(part_age_df['participant_age']=='') & (part_age_df['scraped']==True), 'missing_value'] = True
missing_value_index = part_age_df[part_age_df['missing_value']==True].index.values
part_age_df.loc[missing_value_index -1, 'remove'] = True
part_age_df.loc[(part_age_df['participant_age']=='') & (part_age_df['scraped']==False), 'remove'] = True
part_age_df = deepcopy(part_age_df[part_age_df['remove']!=True].reset_index())
part_age_df.drop(['missing_value', 'remove', 'scraped', 'index'], axis = 1, inplace = True)
# end of added code
part_age_df = pd.DataFrame({'incident_id': part_age_df['incident_id'].iloc[::2].values, 'participant_index': part_age_df['participant_age'].iloc[::2].values, 'participant_age': part_age_df['participant_age'].iloc[1::2].values})
part_age_df['participant_index'] = part_age_df['participant_index'].astype(int)
part_age_df.loc[part_age_df['participant_age']=='', 'participant_age'] = None

# participant status
part_status_df = df.set_index(['incident_id'])['participant_status'].apply(pd.Series).stack().reset_index(level=1, drop=True)
part_status_df = part_status_df.reset_index()
part_status_df.columns = ['incident_id','participant_status']
# added code below due to scraping inconsistency
part_status_df = deepcopy(pd.merge(part_status_df, df[['incident_id', 'scraped']], on = 'incident_id', how='left'))
part_status_df.loc[(part_status_df['participant_status']=='') & (part_status_df['scraped']==True), 'missing_value'] = True
missing_value_index = part_status_df[part_status_df['missing_value']==True].index.values
part_status_df.loc[missing_value_index -1, 'remove'] = True
part_status_df.loc[(part_status_df['participant_status']=='') & (part_status_df['scraped']==False), 'remove'] = True
part_status_df = deepcopy(part_status_df[part_status_df['remove']!=True].reset_index())
part_status_df.drop(['missing_value', 'remove', 'scraped', 'index'], axis = 1, inplace = True)
# end of added code
part_status_df = pd.DataFrame({'incident_id': part_status_df['incident_id'].iloc[::2].values, 'participant_index': part_status_df['participant_status'].iloc[::2].values, 'participant_status': part_status_df['participant_status'].iloc[1::2].values})
part_status_df['participant_index'] = part_status_df['participant_index'].astype(int)

part_df = pd.merge(part_type_df, part_age_df, on=['incident_id', 'participant_index'], how='outer')
part_df = pd.merge(part_df, part_status_df, on=['incident_id', 'participant_index'], how='outer')

part_df['participant_type'] = part_df['participant_type'].astype(str)
part_df['participant_status'] = part_df['participant_status'].astype(str)
part_df['participant_age'] = part_df['participant_age'].astype(float)

part_df.loc[(['Unharmed' in s for s in part_df['participant_status']]), 'num_unharmed'] = 1
part_df.loc[(['Unharmed' not in s for s in part_df['participant_status']]), 'num_unharmed'] = 0
part_df.loc[(['Arrested' in s for s in part_df['participant_status']]), 'num_arrested'] = 1
part_df.loc[(['Arrested' not in s for s in part_df['participant_status']]), 'num_arrested'] = 0
part_df.loc[(['Killed' in s for s in part_df['participant_status']])
            or ((part_df['participant_type']=='Victim') & [s in null_status_victim_killed for s in part_df['incident_id']])
            or ((part_df['participant_type']=='Subject-Suspect') & [s in null_status_suspect_killed for s in part_df['incident_id']]), 'num_killed'] = 1
part_df.loc[(['Killed' not in s for s in part_df['participant_status']]), 'num_killed'] = 0
part_df.loc[(['Injured' in s for s in part_df['participant_status']])
            or ((part_df['participant_type']=='Victim') & [s in null_status_victim_injured for s in part_df['incident_id']])
            or ((part_df['participant_type']=='Subject-Suspect') & [s in null_status_suspect_injured for s in part_df['incident_id']]), 'num_injured'] = 1
part_df.loc[(['Injured' not in s for s in part_df['participant_status']]), 'num_injured'] = 0

In [20]:
part_df.loc[(part_df['incident_id']==1001184) & (part_df['participant_age']==311), 'participant_age'] = 31
part_df.loc[(part_df['incident_id']==926498) & (part_df['participant_age']==209), 'participant_age'] = 33

In [21]:
# assert statement below checks for additional +100 participant_age rows.
# the two that currently exist have been verified manually as accurate.
assert len(part_df[part_df['participant_age'] > 100])==2
assert len(part_df['incident_id'].unique())==len(df['incident_id'].unique())

In [22]:
part_df.to_pickle('../Pickles/part_df.pkl')

In [3]:
part_df = pd.read_pickle('../Pickles/part_df.pkl')

In [9]:
df[df['incident_id']==980595]

,incident_id,date,state,city_or_county,congressional_district,gun_stolen,latitude,longitude,num_guns_involved,participant_age,participant_status,participant_type,state_house_district,state_senate_district,scraped,suicide,accidental,domestic_violence,gang_related,family_related,non_shooting_incident,gun_shop_robbery,drive_by_shooting,officer_involved_shooting,mass_shooting,drug_involved,assault_weapon
2,980595.00,2007-07-16,Rhode Island,Central Falls,1.00,0,41.89,-71.39,1.0,"[0, 42, 1, 27, 2, 2, , 3, 3, ]","[0, Killed, 1, Unharmed,Arrested, 2, 2, , 3, 3, ]","[0, Victim, 1, Subject-Suspect, 2, Subject-Suspect, 3, Subject-Suspect]",56,16,True,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
part_df['participant_status'].unique()

array(['Killed', 'Injured', 'Unharmed,Arrested', '', 'Injured,Arrested',
       'Unharmed', 'nan', 'Arrested', 'Injured, Unharmed, Arrested',
       'Unharmed, Arrested', 'Injured, Arrested',
       'Killed, Unharmed, Arrested', 'Injured, Unharmed',
       'Killed, Injured', 'Killed, Unharmed', 'Killed, Arrested',
       'Injured,Unharmed,Arrested', 'Killed,Unharmed',
       'Killed,Unharmed,Arrested', 'Killed,Injured', 'Killed,Arrested',
       'Injured,Unharmed'], dtype=object)

In [24]:
part_df.tail(3)

,incident_id,participant_type,participant_index,participant_age,participant_status,num_unharmed,num_arrested,num_killed,num_injured
485200,1376626.00,Victim,1,nan,Injured,0.00,0.00,0.00,1.00
485201,1376626.00,Victim,2,nan,Injured,0.00,0.00,0.00,1.00
485202,1376626.00,Victim,3,nan,Injured,0.00,0.00,0.00,1.00


In [25]:
part_df[part_df['incident_id']==95173]

,incident_id,participant_type,participant_index,participant_age,participant_status,num_unharmed,num_arrested,num_killed,num_injured
1936,95173.00,Victim,0,nan,nan,0.00,0.00,0.00,0.00


In [26]:
aggregations_1 = {
    'num_unharmed': 'sum',
    'num_arrested': 'sum',
    'num_killed': 'sum',
    'num_injured': 'sum',
    }

aggregations_2 = {
    'participant_index': 'count',
    'participant_age': 'mean',
    }

part_status_df_group = part_df.groupby(['incident_id']).agg(aggregations_1).reset_index()
part_df_group = part_df.groupby(['incident_id']).agg(aggregations_2).reset_index()
part_type_df_group = part_df.groupby(['incident_id', 'participant_type']).agg(aggregations_2).reset_index()
victim_df = deepcopy(part_type_df_group[part_type_df_group['participant_type']=='Victim'])
suspect_df = deepcopy(part_type_df_group[part_type_df_group['participant_type']=='Subject-Suspect'])

part_df_group.rename({'participant_age': 'avg_participant_age', 'participant_index': 'num_participants'}, axis = 1, inplace = True)
victim_df.rename({'participant_age': 'avg_victim_age', 'participant_index': 'num_victims'}, axis = 1, inplace = True)
suspect_df.rename({'participant_age': 'avg_suspect_age', 'participant_index': 'num_suspects'}, axis = 1, inplace = True)

victim_df.drop(['participant_type'], axis = 1, inplace = True)
suspect_df.drop(['participant_type'], axis = 1, inplace = True)

In [27]:
df = pd.merge(df, part_df_group, on=['incident_id'], how='outer')
df = pd.merge(df, part_status_df_group, on=['incident_id'], how='outer')
df = pd.merge(df, victim_df, on=['incident_id'], how='outer')
df = pd.merge(df, suspect_df, on=['incident_id'], how='outer')

df.loc[df['num_suspects'].isnull(), 'num_suspects'] = 0
df['num_guns_involved'] = df['num_guns_involved'].replace('\\\\t', '', regex = True)

df.drop(['participant_age', 'participant_status', 'participant_type'], axis = 1, inplace = True)

In [28]:
df.to_pickle('../Pickles/df2.pkl')

In [29]:
df = pd.read_pickle('../Pickles/df2.pkl')

In [30]:
len(df)

268677

In [31]:
df.head(3)

,incident_id,date,state,city_or_county,congressional_district,gun_stolen,latitude,longitude,num_guns_involved,state_house_district,state_senate_district,scraped,suicide,accidental,domestic_violence,gang_related,family_related,non_shooting_incident,gun_shop_robbery,drive_by_shooting,officer_involved_shooting,mass_shooting,drug_involved,assault_weapon,num_participants,avg_participant_age,num_unharmed,num_arrested,num_killed,num_injured,num_victims,avg_victim_age,num_suspects,avg_suspect_age
0,1185220.00,1969-12-31,North Carolina,Charlotte,nan,0,nan,nan,1,NaN,NaN,True,0,0,0,0,0,0,0,0,0,0,0,0,1,23.00,0.00,0.00,1.00,0.00,1.00,23.00,0.00,nan
1,1242903.00,1969-12-31,Ohio,Cincinnati,2.00,0,39.20,-84.53,1,33,9,True,0,0,0,0,0,0,0,0,0,0,0,0,1,19.00,0.00,0.00,0.00,1.00,1.00,19.00,0.00,nan
2,980595.00,2007-07-16,Rhode Island,Central Falls,1.00,0,41.89,-71.39,1.0,56,16,True,0,0,0,0,0,0,0,0,0,0,0,0,4,34.50,1.00,1.00,1.00,0.00,1.00,42.00,3.00,27.00


In [32]:
# added las vegas shooting (originally missing from dataset)
df_append = pd.DataFrame(columns = list(df.columns))
# id for las vegas shooting chosing arbitrarily (it's the date of the show)
df_append.loc[0, 'incident_id'] = 100117
df_append.loc[0, 'date'] = pd.Timestamp('2017-10-01')
df_append.loc[0, 'state'] = 'Nevada'
df_append.loc[0, 'city_or_county'] = 'Las Vegas'
df_append.loc[0, 'latitude'] = 36.08833298
df_append.loc[0, 'longitude'] = -115.171499314
df_append.loc[0, 'num_killed'] = 59
df_append.loc[0, 'num_injured'] = 489
df_append.loc[0, 'num_guns_involved'] = 47
df_append.loc[0, 'suicide'] = 1
df_append.loc[0, 'accidental'] = 0
df_append.loc[0, 'domestic_violence'] = 0
df_append.loc[0, 'gang_related'] = 0
df_append.loc[0, 'family_related'] = 0
df_append.loc[0, 'non_shooting_incident'] = 0
df_append.loc[0, 'gun_shop_robbery'] = 0
df_append.loc[0, 'drive_by_shooting'] = 0
df_append.loc[0, 'officer_involved_shooting'] = 1
# df_append.loc[0, 'child_involved'] = 0
df_append.loc[0, 'mass_shooting'] = 1
df_append.loc[0, 'drug_involved'] = 0
df_append.loc[0, 'gun_stolen'] = 0
df_append.loc[0, 'assault_weapon'] = 1
df_append.loc[0, 'num_arrested'] = 0
df_append.loc[0, 'num_suspects'] = 1
df_append.loc[0, 'avg_suspect_age'] = 64
df_append.loc[0, 'avg_victim_age'] = 34.93877551
# giving same value to 20000 participants as the 910 victims because shooting was random
df_append.loc[0, 'avg_participant_age'] = 34.93877551
# number killed + number total injuries from not-only gunfire wikipedia
df_append.loc[0, 'num_victims'] = 910
# total number of attendees at concert
df_append.loc[0, 'num_participants'] = 22000
# attendees who didn't get injured
df_append.loc[0, 'num_unharmed'] = 21090

df = df.append([df_append], sort=False, ignore_index = True)

In [33]:
df['date'] = pd.to_datetime(df['date'])
df['num_killed'] = df['num_killed'].astype(int)
df['num_injured'] = df['num_injured'].astype(int)
df['gun_stolen'] = df['gun_stolen'].astype(int)
df['suicide'] = df['suicide'].astype(int)
df['accidental'] = df['accidental'].astype(int)
df['domestic_violence'] = df['domestic_violence'].astype(int)
df['gang_related'] = df['gang_related'].astype(int)
df['family_related'] = df['family_related'].astype(int)
df['non_shooting_incident'] = df['non_shooting_incident'].astype(int)
df['gun_shop_robbery'] = df['gun_shop_robbery'].astype(int)
df['drive_by_shooting'] = df['drive_by_shooting'].astype(int)
df['officer_involved_shooting'] = df['officer_involved_shooting'].astype(int)
# df['child_involved'] = df['child_involved'].astype(int)
df['mass_shooting'] = df['mass_shooting'].astype(int)
df['drug_involved'] = df['drug_involved'].astype(int)
df['assault_weapon'] = df['assault_weapon'].astype(int)
df['incident_id'] = df['incident_id'].astype(int)
df['num_suspects'] = df['num_suspects'].astype(int)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['num_guns_involved'] = df['num_guns_involved'].astype(float)
df['congressional_district'] = df['congressional_district'].astype(float)
df['avg_participant_age'] = df['avg_participant_age'].astype(float)
df['num_unharmed'] = df['num_unharmed'].astype(float)
df['num_arrested'] = df['num_arrested'].astype(float)
df['num_victims'] = df['num_victims'].astype(float)
df['avg_victim_age'] = df['avg_victim_age'].astype(float)
df['avg_suspect_age'] = df['avg_suspect_age'].astype(float)
df['num_participants'] = df['num_participants'].astype(float)

In [34]:
len(df)

268678

In [35]:
df.head(3)

,incident_id,date,state,city_or_county,congressional_district,gun_stolen,latitude,longitude,num_guns_involved,state_house_district,state_senate_district,scraped,suicide,accidental,domestic_violence,gang_related,family_related,non_shooting_incident,gun_shop_robbery,drive_by_shooting,officer_involved_shooting,mass_shooting,drug_involved,assault_weapon,num_participants,avg_participant_age,num_unharmed,num_arrested,num_killed,num_injured,num_victims,avg_victim_age,num_suspects,avg_suspect_age
0,1185220,1969-12-31,North Carolina,Charlotte,nan,0,nan,nan,1.00,NaN,NaN,True,0,0,0,0,0,0,0,0,0,0,0,0,1.00,23.00,0.00,0.00,1,0,1.00,23.00,0,nan
1,1242903,1969-12-31,Ohio,Cincinnati,2.00,0,39.20,-84.53,1.00,33,9,True,0,0,0,0,0,0,0,0,0,0,0,0,1.00,19.00,0.00,0.00,0,1,1.00,19.00,0,nan
2,980595,2007-07-16,Rhode Island,Central Falls,1.00,0,41.89,-71.39,1.00,56,16,True,0,0,0,0,0,0,0,0,0,0,0,0,4.00,34.50,1.00,1.00,1,0,1.00,42.00,3,27.00


In [36]:
# 2019-04-02 was the maximum on 4/20, beginning of backtrack scrape through original id range
print("Most recent date recorded: " + str(df['date'].dt.date.max()))

before = len(df)
####################################################################################
df = deepcopy(df[df['date'].dt.date <= (df['date'].dt.date.max() - pd.Timedelta(days=1))])
####################################################################################
after = len(df)
print("\n" + str(before-after) + " rows removed after removing most recent scraped date.\n")

before = len(df)
print('Counts of 40 dates removed.')
print(list(df.groupby(df['date'])['incident_id'].count()[-39:].values.astype(int)))
print('')
####################################################################################
df = deepcopy(df[df['date'] <= df.groupby(df['date'])['incident_id'].count()[-40:-39].index[0]])
####################################################################################
after = len(df)
print(str(before-after) + " rows removed after removing most recent 39 days (40 total including removal above).")
print(str(df.groupby(df['date'])['incident_id'].count()[-1]) + " incidents in final incident date.")

Most recent date recorded: 2019-04-20

4 rows removed after removing most recent scraped date.

Counts of 40 dates removed.
[132, 117, 140, 155, 142, 166, 120, 123, 122, 116, 132, 142, 136, 136, 127, 138, 133, 154, 141, 143, 127, 104, 131, 109, 99, 149, 163, 147, 126, 137, 130, 109, 138, 138, 115, 127, 117, 95, 50]

5026 rows removed after removing most recent 39 days (40 total including removal above).
115 incidents in final incident date.


In [37]:
df.columns

Index(['incident_id', 'date', 'state', 'city_or_county',
       'congressional_district', 'gun_stolen', 'latitude', 'longitude',
       'num_guns_involved', 'state_house_district', 'state_senate_district',
       'scraped', 'suicide', 'accidental', 'domestic_violence', 'gang_related',
       'family_related', 'non_shooting_incident', 'gun_shop_robbery',
       'drive_by_shooting', 'officer_involved_shooting', 'mass_shooting',
       'drug_involved', 'assault_weapon', 'num_participants',
       'avg_participant_age', 'num_unharmed', 'num_arrested', 'num_killed',
       'num_injured', 'num_victims', 'avg_victim_age', 'num_suspects',
       'avg_suspect_age'],
      dtype='object')

In [38]:
aggregations = {
    'incident_id': 'count',
    'num_killed': 'sum',
    'num_injured': 'sum',
    'gun_stolen': 'sum',
    'num_guns_involved': 'sum',
    'suicide': 'sum',
    'accidental': 'sum',
    'domestic_violence': 'sum',
    'gang_related': 'sum',
    'family_related': 'sum',
    'non_shooting_incident': 'sum',
    'gun_shop_robbery': 'sum',
    'drive_by_shooting': 'sum',
    'officer_involved_shooting': 'sum',
#     'child_involved': 'sum',
    'mass_shooting': 'sum',
    'drug_involved': 'sum',
    'assault_weapon': 'sum',
    'num_unharmed': 'sum',
    'num_arrested': 'sum',
    'num_suspects': 'sum',
    'avg_suspect_age': 'mean',
    'num_victims': 'sum',
    'avg_victim_age': 'mean',
    'num_participants': 'sum',
    'avg_participant_age': 'mean',
    }

df_states = df.groupby(['state', 'date']).agg(aggregations).reset_index()
df_states.rename({'incident_id': 'num_incidents'}, axis = 1, inplace = True)
df_date = df.groupby(['date']).agg(aggregations).reset_index()
df_date.rename({'incident_id': 'num_incidents'}, axis = 1, inplace = True)

In [39]:
df_states.to_pickle('../Pickles/df_states.pkl')
df_date.to_pickle('../Pickles/df_date.pkl')

In [40]:
df_states = pd.read_pickle('../Pickles/df_states.pkl')
df_date = pd.read_pickle('../Pickles/df_date.pkl')
state_laws_df = pd.read_pickle('../Pickles/state_firearm_laws.pkl')

In [41]:
df_states = deepcopy(df_states[df_states['date'].dt.year >= 2014].reset_index())
df_states.drop('index', axis = 1, inplace = True)
df_date = deepcopy(df_date[df_date['date'].dt.year >= 2014].reset_index())
df_date.drop('index', axis = 1, inplace = True)

In [42]:
df_states.drop(['avg_victim_age', 'avg_suspect_age',
                'num_participants', 'avg_participant_age'],
               axis = 1, inplace = True)
df_date.drop(['avg_victim_age', 'avg_suspect_age', 'num_participants',
              'avg_participant_age'],
             axis = 1, inplace = True)

In [43]:
missing_dates = set(pd.period_range(min(df_date['date']), max(df_date['date']))) - set(list(pd.DatetimeIndex(df_date['date']).to_period('D')))
print("{} missing dates across the US.".format(len(missing_dates)))
df_append = pd.DataFrame(list(missing_dates), columns = ['date'])
df_append['date'] = pd.to_datetime(df_append['date'].astype(str))
for column in df_date.columns:
    if column != 'date':
        df_append[column] = 0
df_date = df_date.append(df_append, sort=False, ignore_index = True)

for state in df_states['state'].unique():
    missing_dates = set(pd.period_range(min(df_states['date']), max(df_states['date']))) - set(list(pd.DatetimeIndex(df_states[df_states['state']==state]['date']).to_period('D')))
    print("{} missing dates in {}.".format(len(missing_dates), state))
    df_append = pd.DataFrame(list(missing_dates), columns = ['date'])
    df_append['date'] = pd.to_datetime(df_append['date'].astype(str))
    df_append['state'] = state
    for column in df_states.columns:
        if column != 'date' and column != 'state':
            df_append[column] = 0
    df_states = df_states.append(df_append, sort=False, ignore_index = True)

0 missing dates across the US.
99 missing dates in Alabama.
1008 missing dates in Alaska.
536 missing dates in Arizona.
359 missing dates in Arkansas.
0 missing dates in California.
373 missing dates in Colorado.
357 missing dates in Connecticut.
737 missing dates in Delaware.
532 missing dates in District of Columbia.
0 missing dates in Florida.
25 missing dates in Georgia.
1602 missing dates in Hawaii.
1301 missing dates in Idaho.
4 missing dates in Illinois.
70 missing dates in Indiana.
541 missing dates in Iowa.
560 missing dates in Kansas.
190 missing dates in Kentucky.
20 missing dates in Louisiana.
1204 missing dates in Maine.
84 missing dates in Maryland.
110 missing dates in Massachusetts.
82 missing dates in Michigan.
654 missing dates in Minnesota.
230 missing dates in Mississippi.
71 missing dates in Missouri.
1351 missing dates in Montana.
819 missing dates in Nebraska.
671 missing dates in Nevada.
1153 missing dates in New Hampshire.
111 missing dates in New Jersey.
812 m

In [44]:
assert len(list(set(df_states['state'].value_counts().values)))==1

In [45]:
for column in df_states.columns:
    if column=='date':
        df_states['date'] = pd.to_datetime(df_states['date'])
    elif column=='state':
        pass
    else:
        df_states[column] = df_states[column].astype(int)
        
for column in df_date.columns:
    if column=='date':
        df_date['date'] = pd.to_datetime(df_date['date'])
    else:
        df_date[column] = df_date[column].astype(int)

In [46]:
df_states['year'] = df_states['date'].dt.year
df_states = pd.merge(df_states, state_laws_df, on=['state', 'year'], how='left')
df_states = df_states.reset_index()
df_states.drop(['index'], axis = 1, inplace = True)

df_states = df_states.sort_values(by = 'date', ascending = True)
df_date = df_date.sort_values(by = 'date', ascending = True)

In [47]:
assert len(df_date)==len(df_states)/51

In [48]:
df_states.to_pickle('../Pickles/df_states_2.pkl')
df_date.to_pickle('../Pickles/df_date_2.pkl')

In [49]:
df_states = pd.read_pickle('../Pickles/df_states_2.pkl')
df_date = pd.read_pickle('../Pickles/df_date_2.pkl')

In [50]:
print(len(df_date[df_date['num_incidents']==0]))
print(len(df_states[df_states['num_incidents']==0]))

0
26327
